**Содержание**<a id='toc0_'></a>    
- [Основные виды нейросетевых моделей для обработки текстов](#toc1_)    
    - [Виды и роли разлиных нейросетевых `модулей`](#toc1_1_1_)    
- [Свёрточные нейросети](#toc2_)    
      - [Пример](#toc2_1_1_1_)    
      - [Применение свертки к тексту](#toc2_1_1_2_)    
      - [Пример](#toc2_1_1_3_)    
      - [Пример](#toc2_1_1_4_)    
      - [Пример](#toc2_1_1_5_)    
      - [Пример](#toc2_1_1_6_)    
  - [Некоторые выводы](#toc2_2_)    
      - [Пример](#toc2_2_1_1_)    
      - [Пример](#toc2_2_1_2_)    
- [Выводы](#toc3_)    
  - [Выбор архитектуры нейронных сетей для задач NLP](#toc3_1_)    
- [Теоретические вопросы: Свёрточные нейросети в обработке текста](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Основные виды нейросетевых моделей для обработки текстов](#toc0_)

Cмысл использования нейросетей — в том, что они могут выделить сложные паттерны — взаимоотношения в данных. 

В случае текстов (а именно, отдельных предложений) такие паттерны проявляются в том, как именно слова употребляются вместе, как построены фразы. Другими словами, нейросети нужны для того, чтобы представить контекст в виде математического объекта — вектора или матрицы. 

Ранее мы уже говорили о первоначальном переводе текста в матрицу с помощью эмбеддингов и методов дистрибутивной семантики. 

Нейросети выполняют следующий шаг: 
- преобразование этой матрицы так, чтобы каждая строчка содержала информацию не об отдельных словах, а о фразах и их смыслах. 

### <a id='toc1_1_1_'></a>[Виды и роли разлиных нейросетевых `модулей`](#toc0_)

Задачи обработки текста являются комплексными и инструменты их решения подразделяются на крупные блоки методов из которых собирается "конвеер", решающий конкретную прикладную задачу.

Такие блоки методов (модули) включают:

- свёрточные блоки (convolutional NN, CNN)
    - аналогичны работающим с изображениями (используются не двухмерные свёртки, а одномерные)
    - выявляют паттерны вне зависимости от их позиции (инвариантность к переносу в пространстве)
    - быстные, простые, хорошо обучаются (хорошо распрараллеливаются)
    - могут быть недостаточно гибкими (не могут сравнивать информацию о словах в начале и конце предложения, увеличение размера изучаемого паттерна требует кратного увеличения количества параметров)

- рекуррентные блоки (recurrent NN, RNN)
    - лучше подходят к задачам обработки текста
    - читаем слово за словом или символ за символом и на каждом шаге поддерживаем некоторый вектор:
        - **вектор скрытого состояния**, который содержит информацию о всём тексте, который мы прочитали ранее
    - может учитыват достаточно длинные зависимости
    - длина учитываемых зависимостей не обязательно связана с количеством параметров сети
    - учить рекуррентные сети гораздо сложнее.

        *В целом нейросетевые блоки обеспечивают:*  
        *- Учет сложного контекста слов*  
        *- Улучшение качества решения задачи за счёт объединения нескольких шагов в одну архитектуру, обучаемую end-to-end *градиентным спуском*  
        *- Анализ формулировок, структуры фраз*

- блоки объединения (агрегации, [pooling](https://en.wikipedia.org/wiki/Convolutional_neural_network#Pooling_layer))
    - аналогичные соответствующим методам обработки изображений (оставляют главное, убирают незначащие детали), в обработки изображений, например, усреднение матрицы изображений по соседним элементам и т.п.
    - локальный пулинг - уменьшить длину, укрупнить детали
    - глобальный пулинг - например, для задачи классификации текстов, по векторам токенов и т.п. получить вектор текста фиксированного размера

- механизм внимания (attention mechanism, self-attention) - один из свежих подходов
    - оказалось, что достаточно лишь одного "внимания" для решения почти всех задач обработки текстов
    - учитывает сколь угодно далекие друг от друга зависимости
    - умная агрегация
    - сети с вниманием быстрые, хорошо обучается
    - можно рассматривать механизм внимания как умный адаптивный пулинг

- архитектуры с памятью (memory NN, neural Turing machines) - считается экзотикой
    - обобщение рекуррентных нейросетей (в RNN один вектор в ячейке, здесь много таких векторов, и на каждом шаге мы можем выбрать)
    - потенциально (теоретически) могут решать абсолютно любую задачу
    - на практике медленно сходятся, полезная емкость памяти часто оказывается меньше чем ожидается
  
- рекрусивные сети (tree-recursive NN)
    - еще одно обобщение RNN, но не путем добавлением памяти, а за счет использовая деревьев, а не последовательностей (токенов и т.п.)
    - т.е. сначала строиться некоторое семантическое дерево, а потом из него, а не из текста набирается информация для нейросети
    - полезно для языков со свободным порядком слов (в т.ч. русского)
    - однако, на практике часто результативность не сильно лучше RNN

- графовые сверточные нейросети (graph convolutionsl NN, GCNN)
    - обобщение RNN+CNN на произвольную структуру графа
        - в RNN в тексте в картинках токены/пикесели связаны только с соседями (одномерно, двухмерно), а тут некоторым образом строится граф связей элементов текста/картинки
    - новое перспективное направлени
    - высокая вычислительная сложность

# <a id='toc2_'></a>[Свёрточные нейросети](#toc0_)

Cвёртка - операция, выполняемая над двумя функциями или сигналами, дающая некий третий сигнал. 
$$(f*g)(x) = \int_{-\infin}^{\infin} f(x) g(x -y)dy$$
$$ConvOut_i = \sum_{k-1}^K InSignal_{i-\frac{K}{2}+k} \cdot Kernel_k, \forall i \in 0,1, ...L$$


- первый сигнал ($f$) — это входные данные, 
- второй сигнал ($g$) — это bias, intercept (часть параметров нейросети, которые выполняют роль **ядра свёртки**)
- перебираются все возможные смещения ядра относительно входящего сигнала
- для каждого смещения измеряется сходство фрагмента сигнала с ядром
    - скалярное произведение фрагмента сигнала с ядром свёртки для дискретного случая

В дискретном случае:
- результат применения одномерной свёртки — это вектор. 
    - допустим есть входной сигнал размера L+K-1. 
    - допустим ядро имеет размер K = 3
    - проходя по сигналу окном размера K получим L значений скалярного произведения ядра и фрагментов сигнала
- очевидно, что фиксированное ядро с одинаковыми фрагментами сигнала даст одинаковую свертку, независимо от расположения этого фрагмента в сигнале

Это позволяет нам выделять локальные паттерны безотносительно их позиции в сигнале. А **ядро свёртки**, при этом, описывает паттерн, который мы ищем.

#### <a id='toc2_1_1_1_'></a>[Пример](#toc0_)

Примените свёртку с ядром (-0.5, 0, 0.5) к сигналу (1, 1, 2, 3, 3, 3, 2, 1, 1). Ответ запишите в виде последовательности чисел, разделённых пробелами. В качестве десятичного разделителя используйте точку. Входную последовательность не нужно дополнять фиктивными элементами (padding выключен).

Шаг свёртки (stride в PyTorch) считаем 1.

    - сигнал симметричный
    - свертка показала противоположные значения на зеркально симметричных фрагментах сигнала
    - потомушта такое ядро специальное 

In [1]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view     # numpy>=1.20

sig = np.array([1,1,2,3,3,3,2,1,1])
ker = np.array([-0.5, 0, 0.5])
fragments = sliding_window_view(sig, window_shape = ker.shape[0])
fragments @ ker

array([ 0.5,  1. ,  0.5,  0. , -0.5, -1. , -0.5])

In [2]:
np.convolve(sig, ker, 'valid')

array([-0.5, -1. , -0.5,  0. ,  0.5,  1. ,  0.5])

In [3]:
import torch
import torch.nn as nn

# размерность тензора должна соответствовать явно: (1, 1, 3), а не (3, )
# тип должен совпадать с типом в ядра, т.к. торч это Си
input_1d = torch.tensor(sig.reshape(1,1,-1).astype('float'))    
kernel = torch.tensor(ker.reshape(1,1,-1))

nn.functional.conv1d(input_1d, kernel, padding=0)   # 

tensor([[[ 0.5000,  1.0000,  0.5000,  0.0000, -0.5000, -1.0000, -0.5000]]],
       dtype=torch.float64)

#### <a id='toc2_1_1_2_'></a>[Применение свертки к тексту](#toc0_)

- строится матрица эмбеддингов (дистрибутивная семантика) (Количество токенов Х Длина эмбеддинга)
- проходим по матрице окном размера $k$ (строк/эмбеддингов)
- вытягиваем эти $k$ эмбеддингов в один вектор
- вычисляем скалярное произведение с ядром (его размерность: Величина окна Х Длина эмбеддинга)
    - тоже самое делается с другими ядрами (ядер будет много, каждое ищет свой паттерн)
    - получаем вектор для окна размерности, равной количеству ядер
- в итоге получили матрицу размерности ((Количество окон = Длина текста - Размер окна + 1) Х Количество ядер)

- следующий возможный шаг - применение к этой матрице блока пулинга, который более активно, чем свертка, уменьшает число строк в матрице (усредняет соседние строки, или еще как то)

    - Обычно свёрточные блоки применяют один за другим, перемешивая (это пока не очень понятно, походу имеется в виду, что просто в конвеер подключается пулинг) с блоками пулинга или агрегации, а также с функциями активации. Делают это для того, чтобы расширить пятно восприятия, то есть получить возможность обрабатывать более широкий контекст, более длинные паттерны: вектор, выходящий из пулинга уже зависит от большего числа входящих эмбеддингов (**рецептивное поле**, receptive field)

**Вопрос:**

В картиночных задачах каждый элемент входного вектора имеет интерпретацию, то можно сделать такие любопытные вещи как, обучить сеть, взять значение промежуточного слоя, использовав его как маску, наложить на входное изображение и получить области на исходном изображении, которые наиболее важны для нейронной сети, а можно ли как-то сделать тоже самое, но для задач НЛП?

**Ответ:**

В разных архитектурах для этого используются "слои внимания" (attention layers), и в современных архитектурах они позволяют не просто узнать, на что смотрела сеть (ради любопытства или оптимизации), но направить обработку следующих слоёв. То есть слой внимания вырабатывает дополнительный набор признаков, который конкатенируется или прямо домножается на векторное представление (слова, фразы, параграфа), по которому следующие слои учатся обрабатывать текст согласно заданию.

**Воспрос:**

Свертка в задачах НЛП не такая очевидная, как в картиночных задачах. Ведь в картиночных задачах каждый элемент вектора имеет физический смысл вида, яркости пикселя в RGB пространстве (или другом), а в задачах НЛП же, значение вектора это уже некоторая абстракция, которая напрямую никак не сопоставляется с каким-то словом или символом.

**Ответ:**

Промежуточное векторное описание до каких-то пор тоже не очень специфично отражает смысл слова. До тех пор, пока не появился word2vec и стало возможным составлять смысловые пропорции типа "огурец минус зелёный плюс фиолетовый"... Свёртка просто делает что-то с числами, векторным представлением слов/предложений/параграфов, и если представление построено согласно адекватной модели языка, состав масок свёртки может означать что-то интересное.


#### <a id='toc2_1_1_3_'></a>[Пример](#toc0_)
Оцените ширину рецептивного поля для нейросети, состоящей из 4 свёрточных слоёв с ядром 5, соединённых последовательно. Код на PyTorch для создания такой нейросети:

    nn.Sequential(
        nn.Conv1d(in_channels=1, out_channels=1, kernel_size=5),
        nn.Conv1d(in_channels=1, out_channels=1, kernel_size=5),
        nn.Conv1d(in_channels=1, out_channels=1, kernel_size=5),
        nn.Conv1d(in_channels=1, out_channels=1, kernel_size=5)
    )

Ширина рецептивного поля - наибольшее расстояние между элементами входной последовательности, которые могут влиять на один и тот же элемент выходной последовательности, плюс один.

Например, ширина рецептивного поля для одного свёрточного блока с ядром 3 равна 3. Для двух таких блоков, соединённых последовательно - 5.

На каждом шаге (сверточном слое) прибавляется справа и слева по половине окна без центра, плюс сам центр окна:

$$rf_1 = k = 1 + \frac{k-1}{2} \cdot 2 \cdot 1 \\
rf_2 = 1 + \frac{k-1}{2} \cdot 2 \cdot 2 \\
rf_3 = 1 + \frac{k-1}{2} \cdot 2 \cdot 3 \\
... \\
rf_n = 1 + (k-1) \cdot n $$


In [4]:
k, n = 5, 4
rf = 1 + (k-1)* n
rf

17

#### <a id='toc2_1_1_4_'></a>[Пример](#toc0_)

**Примените свёртку к входной последовательности, в которой каждый элемент кодируется вектором размерности 2 (часть 1)**

Рассмотрим многоканальную свертку, принимающую на вход матрицу $X^{InLen, InChannels}$, где $InLen$ - количество строк, равное длине входной последовательности, $InChannels$ - количество столбцов, равное количеству признаков для каждого элемента во входной последовательности.

В результате получается матрица $Y^{OutLen}$, где $OutLen = InLen - K + 1$ - длина выходной последовательности, $K$ - размер ядра свёртки.

Ядро свертки с одним выходным каналом задаётся двумерным тензором $Kernel^{InChannels, K}$.

Многоканальная свёртка реализуется следующей формулой (аналогично одноканальной, но на выходной канал влияют все входные каналы):

$$Y[pos] = Bias+ \sum_{k=0}^{K-1} \sum_{ic=0}^{InChannels - 1} X_{[pos + k,ic]} Kernel_{[ic,k]}$$
 
Обратите внимание, что по аналогии с логистической регрессией, кроме ядра свёртки, есть ещё обучаемый параметр сдвига $Bias$.

Рассмотрим конкретный пример:

<img src="./img/convo1_color.png" width="300">


 

Для входной последовательности:

$$X^{5, 2} = \left( \begin{matrix} 1 & 0 \\ 1 & 1 \\ 0 & 0 \\ 0 & 1 \\ 1 & 0 \\ \end{matrix} \right)$$

Ядра, заданного следующей матрицей и нулевого смещения:

$$Kernel^{2, 3}= \left( \begin{matrix} 1 & 1 & 0\\ 0 & 1 & 1\\ \end{matrix} \right) , Bias=0$$

Посчитайте выходной вектор Y:

$$Y[pos] = Bias + \sum_{k=0}^{2} \sum_{ic=0}^{1} X_{[pos + k,ic]} Kernel_{[ic, k]}$$

В ответ впишите содержимое вектора $Y$ , состоящего из трех чисел, числа разделяйте пробелами. В качестве разделителя дробной части используйте точку.

Параметр сдвига считайте нулевым.

In [5]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view         # numpy>=1.20

x = np.array([[1, 1, 0, 0, 1], 
              [0, 1, 0, 1, 0]]).T
ker = np.array([[1, 1, 0], 
                [0, 1, 1]])
b = 0

windows = sliding_window_view(x, window_shape = ker.T.shape)    # лишнее измерение схлопнется в сумме
masked = windows * ker.T
b + np.sum(masked, axis=tuple(range(1, masked.ndim)))           # сумма по всем измерениям, кроме первого

array([3, 2, 1])

In [6]:
from scipy.signal import convolve2d

convolve2d(x.T, ker, mode='valid').squeeze()                    # в 3 раза шустрее

array([3, 2, 1])

#### <a id='toc2_1_1_5_'></a>[Пример](#toc0_)
**Примените свёртку к входной последовательности, в которой каждый элемент кодируется вектором размерности 2. (часть 2)**

Усложним еще немного многоканальную свертку: теперь у нас будет несколько выходных каналов.

По прежнему будем подавать на вход матрицу $X^{InLen, InChannels}$, где $InLen$ - количество строк, равное длине входной последовательности, $InChannels$ - количество столбцов, равное количеству признаков для каждого элемента во входной последовательности.

Но на этот раз ядро многоканальной свёртки задается трёхмерным тензором $Kernel^{OutChannels, InChannels, K}$, где новым измерением будет $OutChannels$ - количество признаков для каждого элемента выходной последовательности.

Тогда результатом применения свертки будет матрица $Y^{OutLen, OutChannels}$, где $OutLen = InLen - K + 1$ - длина выходной последовательности, $K$ - размер ядра свёртки, $OutChannels$ - количество признаков для каждого элемента выходной последовательности.

Свёртка реализуется следующей формулой:

$Y[pos,oc] = Bias[oc] + \sum_{k=0}^{K-1} \sum_{ic=0}^{InChannels - 1} X[pos + k,ic] Kernel[oc,ic,k]$

Обратите внимание,  теперь параметр сдвига - это вектор обучаемых параметров $Bias^{OutChannels}$ 

Рассмотрим модифицированный пример из прошлого шага:

<img src="./img/convo2_color.png" width="300">

Входная последовательность:

$$X^{5, 2} =\left( \begin{matrix} 1 & 0 \\ 1 & 1 \\ 0 & 0 \\ 0 & 1 \\ 1 & 0 \\ \end{matrix} \right)$$

Ядро (каждая матрица соответствует срезу по первому измерению - $OutChannels$, - и имеет семантику $InChannels x KernelSize$:

$$Kernel^{2,2, 3}=\left( \begin{matrix} 1 & 1 & 0\\ 0 & 1 & 1\\ \end{matrix} \right) \left( \begin{matrix} 1 & 0 & 0\\ 0 & 0 & 1\\ \end{matrix} \right)$$

В ответ впишите содержимое матрицы, каждая строка матрицы на отдельной строке, числа разделять пробелами. В качестве разделителя дробной части использовать точку.

Параметр сдвига считайте нулевым.

In [7]:
x = np.array([[1, 1, 0, 0, 1], 
              [0, 1, 0, 1, 0]])

ker = np.array([[[1, 1, 0], 
                 [0, 1, 1]], 
                [[1, 0, 0], 
                 [0, 0, 1]],
                [[1, 0, 0], 
                 [0, 1, 1]]])

def convolve_Nto1(x, ker, b=0):
    windows = sliding_window_view(x.T, window_shape = ker.T.shape)
    masked = windows * ker.T
    return b + np.sum(masked, axis=tuple(range(1, masked.ndim)))

def convolve_NtoM(x, ker, b=0):
    return np.array(list(map(lambda ker: convolve_Nto1(x, ker, b=b), ker))).T

convolve_NtoM(x, ker)

array([[3, 1, 2],
       [2, 2, 2],
       [1, 0, 1]])

#### <a id='toc2_1_1_6_'></a>[Пример](#toc0_)
Оцените количество параметров в свёрточном блоке, принимающем 64 канала, возвращающем 128 каналов, длина ядра 5. Не забывайте про параметры сдвига (bias, их количество соответствует количеству выходных каналов).

Че такое параметры? То что мы оцениваем в ходе вычисления блока:

- Количество входных каналов Х Размер ядра Х Количество выходных каналов + Количество выходных каналов (смещения) = 41088



## <a id='toc2_2_'></a>[Некоторые выводы](#toc0_)

- Как правило, свёрточные нейросети хорошо обучаются и сходятся. 
  - слабо выражена проблема **затухания градиента**
- Такие архитектуры хорошо подходят для распознавания коротких локальных особенностей, например — словосочетаний или коротких фраз. 
- Заставить свёрточную нейросеть обрабатывать длинные предложения и учитывать широкий контекст достаточно сложно 
  - для этого необходимо сделать очень глубокую сеть, в которой неминуемо будет много параметров, что сильно усложнит процесс обучения
  - максимальная ширина паттерна привязана к количеству параметров нейросети 

- Чтобы расширить пятно восприятия, можно применять "разреженные свёртки" ("dilated convolutions"). 
  - Идея в том, что ядро свёртки применяется не к непрерывному фрагменту сигнала, а к фрагменту, из которого удалена часть элементов (как правило, удаляют элементы с чётными номерами). Таким образом почти в два раза увеличивается рецептивное поле, а количество параметров остаётся прежним. 
  - Если применять разреженные свёртки на первом слое, мы, фактически, будем игнорировать каждое второе слово, это совсем не то что мы хотим. Поэтому на первом слое применяют обычные, не прореженные свёртки, а затем на каждом новом уровне увеличивают прореживания в два раза. Таким образом, мы можем сэкономить количество параметров, увеличив рецептивное поле. 

- применимы для задачи генерации текста
  - для вычисления свёртки нам требуются элементы не только слева, но и справа. Для решения задачи генерации текста, берется только левая часть окна, по которму проходит ядро. Такой подход, кстати, называется **авторегрессией**.

В свёрточных нейросетях проблема **затухания градиента** тоже есть. Однако она проявляется не с ростом длины входной последовательности, а с увеличением глубины (то есть количества слоёв). 

Самые распространённые методы борьбы с затуханием градиента - частично-линейные функции активации (например, LearkyReLU), блоки со связями в обход нелинейностей (skip connections, residual blocks), а также нормализация (например, BatchNorm).

#### <a id='toc2_2_1_1_'></a>[Пример](#toc0_)
Вычислите, сколько потребуется свёрточных слоёв с непрореженными свёртками (dilation=1 в PyTorch) с ядром длины 5, чтобы учесть связь между первым словом в предложении и 30-м.

$$ rf_n = 1 + (k-1) \cdot n \\
30 = 1 + (5 - 1) \cdot n \\
n = \frac{30 - 1}{5 - 1} = 7.25 \rightarrow 8$$

#### <a id='toc2_2_1_2_'></a>[Пример](#toc0_)

Вычислите, сколько потребуется свёрточных слоёв с прореженными свёртками (dilation>1 в PyTorch) с ядром длины 5, чтобы учесть связь между первым словом в предложении и 30-м (при этом длина текста больше 30и слов!).

Начинать принято с dilation=1 (непрореженные свёртки) и увеличивать на каждом шаге в 2 раза: $Dilation[layer]=2^{layer}$, layer - номер слоя, **начиная с 0**.

$$ rf_n = 1 + (k-1) \cdot 2^{n}\\
30 = 1 + (5 - 1) \cdot 2^n\\
2^n = \frac{30 - 1}{5 - 1} = 7.25\\
\ln 2^{n} = \ln 7.25 = 1.981...\\
n = 2.85...\\
n = 3$$

Но тут n это номер слоя, начиная с 0, поэтому количество слоев 4

In [8]:
rf = lambda n: 1 + (k - 1) * 2 ** n
list(map(rf, range(5))), np.log(7.25) / np.log(2)

([5, 9, 17, 33, 65], 2.8579809951275723)

Общая формула количества слов на выходе слоя от количества слов на входе:

$$W_{out} = \left \lfloor \frac{W_{in} + 2 \times padding - dilation \times (ker\_size - 1)}{stride} + 1 \right \rfloor$$

    padding - добавление справа и слева фиктивных слова
    dilation - прореживание (сколько пропускается)
    ker_size - размер ядра свёртки
    stride - шаг свертки
Смысл этих параметров хорошо показан [тут](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md)

1. Формула используется для согласованого выбора параметров свертки под нужные значения количества вх./вых. каналов
2. Для многомерной (многоканальной) свертки также верна, с учетом того, что по каждому $i$-му измерению будут свои количества входов/выходов, и выбираются свои $padding_i$, $dilation_i$, $ker\_size_i$, $stride_i$.

# <a id='toc3_'></a>[Выводы](#toc0_)
1. Одна из проблем свёрток заключается в том, что максимальная ширина паттерна привязана к количеству параметров нейросети. Это не очень удобно. 
2. Увеличить рецептивное поле можно с помощью блоков агрегации или пулинга, а также с помощью прореживания. 
3. Авторегрессионные модели — это что-то среднее между свёрточными и рекуррентными нейросетями.

## <a id='toc3_1_'></a>[Выбор архитектуры нейронных сетей для задач NLP](#toc0_)

Как обычно **на практике** подходят к решению задач с помощью нейронный сетей те, кто этим реально занимается (Роман Суворов, автор курса):

1. Если для этой задачи уже кто-то что то применял, надо взять основные идеи (принять за **baseline**). Читать статьи и чужой код в поисках идей.
2. Постараться выяснить, **какие особенности языка могут отвечать за решение задачи** (это отдельные фразы или лексический состав в целом или что-то ещё, насколько далёкие связи нужно уметь улавливать). Нужно постараться заложить своё понимание механизма (как это происходит в реальном мире) в архитектуру (модель реального мира). Например, если целевые метки сильно коррелируют с общим составом словаря, не нужно пытаться городить свёртки с большим receptive field, и вообще не надо сильно усложнять архитектуру - линейная модель + fasttext должны зарешать (если речь о классификации). Наоборот, если важен контекст, нужно постараться максимально быстро вырастить рецептивное поле за наименьшее число слоёв. Закладывание знаний и ограничений в модель ещё называется inductive bias - модели будет проще обобщиться, если данных не сильно много.
3. Когда сделал вариант архитектуры, добиться, чтобы она начала переобучаться на ограниченном количестве данных (например, на одном батче). Когда она сможет переобучиться - значит при большем количестве данных она сможет и обобщиться. **Если модель не переобучается на одном батче, значит она вообще не учится.**
4. Сравнивать **train loss и validation loss** - если на трейне лосс сильно меньше, то мы переобучаемся, надо облегчать модель (добавить дропаута, уменьшить количество каналов в свёртках, добавить аугметаций). Если на трейне и валидации лосс одинаковый, то мы недообучаемся, надо усилить модель. Если на валидации лосс сильно меньше, то скорее всего валидационный датасет либо проще, либо валидация вообще нерепрезентативная - надо внимательно на неё посмотреть - чудес не бывает. Если датасет маленький, важно использовать кросс-валидацию или усреднение по нескольким случайным разбивкам на трейн и тест.
5. Если хочешь **увеличить количество слоёв**, используй residual connections (в курсе они ещё называются "связи в обход нелинейностей") - это сильно упрощает сходимость.
6. **Cвёрточные НН** проще заставить работать, чем рекуррентки, поэтому лучше начинать с них.
7. Можно **комбинировать** разные виды архитектур - сначала сделать несколько слоёв свёрток, потом добавить рекуррентку, сверху добавить, например, условные случайные поля (CRF), если задача типа NER, Named-entity recognition (выделить спаны сущностей (персоны, локации, даты и т.п) в тексте).
8. Сначала стоит **добиваться наилучшего качества** любыми средствами - прикручивая сколь угодно тяжелые модели, берты, трансформеры и т.п. Когда качество устраивает - можно переключиться на сжатие полученной модели.
9. Смотреть глазами на ошибки, **генерировать гипотезы** насчёт "почему эта модель совершает именно эти ошибки", вносить изменения в архитектуру **и проверять их экспериментально** - не надо слишком много теоретизировать.
10. Ну и в целом **идти от простого к сложному**, за исключением случаев, когда точно по опыту знаешь, что простое не сработает. Когда есть мысль что-то добавить в архитектуру, спрашивать себя "можно ли без этого" или "есть ли что-то более простое, что можно попробовать".

[Хороший сборник решений разных задач по NLP](https://github.com/microsoft/nlp-recipes)
    

Семинар в директории `./stepik-dl-nlp`

# <a id='toc4_'></a>[Теоретические вопросы: Свёрточные нейросети в обработке текста](#toc0_)

Напишите функцию, применяющую свёрточный модуль к последовательности. 

In the simplest case, the output value of the layer with input size $(N, C_{\text{in}}, H, W)$ and output $(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ can be precisely described as:

$$\text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) + \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)$$

where $\star$ is the valid 2D cross-correlation operator (да короче свертка это), $N$ is a batch size, $C$ denotes a number of channels, $H$ is a height of input planes in pixels, and $W$ is width in pixels.

**В проверялке numpy==1.14.1 (`np.lib.stride_tricks.sliding_window_view` отсутствует)**
- используем фишку из [готового алгорима](https://towardsdatascience.com/fast-and-robust-sliding-window-vectorization-with-numpy-3ad950ed62f5), чтобы получать скользящее окно без цикла:
  - в основе - свойства broadcasting (автоматическое расширение размерности) numpy
  - строится хитрая матрица индексов для одного окна
  - к ней добавляется размерность, из-за которой целевая матрица нужным образом расширяет свою размерность
- остальное тривиально (не совсем)
  - приведение тензоров к совместимому виду
  - тензорное умножение (на самом деле тензорная редукция)
  
**Получилось просто красиво и исключительно быстро**

In [9]:
import sys
import ast
import numpy as np

SAMPLES = """[[0.6685863697825855, 0.9865099796400376], [0.32297881307708, 0.9908870158650515], [0.8359169063921157, 0.5443776713017927], [0.5363888029267118, 0.34755850459471804], [0.5966372342560426, 0.9834742307894673], [0.7371274295314912, 0.03279590013405109], [0.08580648148402137, 0.2850655085982621], [0.584942134340805, 0.7981720699451806], [0.19604496972304086, 0.991819073359733], [0.5622511488322055, 0.07928952553499002], [0.24152151330089744, 0.2865696384305756], [0.882594506643994, 0.5949729821472712], [0.10820233432771786, 0.8549971123651271], [0.18754460128195194, 0.6303661925298489], [0.3551051497971416, 0.9452980688158904], [0.6525044770663634, 0.8054232618991838]]\n[[[0.8638059436915633, 0.4002290439648929, 0.8174398057982054, 0.34082478315585973, 0.5565832130592809], [0.08497737591188492, 0.7853885140384725, 0.1645895575029136, 0.6294907704137637, 0.8169862258229014]], [[0.21527072709338957, 0.9185427760457524, 0.5167378860756242, 0.12177789993763499, 0.4201289643444214], [0.8389450071463863, 0.6238637143288427, 0.5098771768082815, 0.1436853463091461, 0.12036561608743845]], [[0.8347050184618267, 0.12339875692133984, 0.13629086964943626, 0.623950910768403, 0.7092189295761365], [0.9578703072530402, 0.31612669923975534, 0.44018179806384916, 0.26615330385390035, 0.2745979551030847]]]\n[0.6574440445518804, 0.3253787051567585, 0.3119672663686287]""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать

def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def apply_convolution_trace(data, kernel, bias):
    kernel_product = data @ kernel
    res = [np.trace(kernel_product, pos, 2) for pos in range(data.shape[0] - kernel.shape[2] + 1)]

    return res + bias

def apply_convolution_looper(data, kernel, bias):
    res = np.zeros([len(data)-kernel.shape[2]+1,len(kernel)])
    for n in range(kernel.shape[0]):
        for i in range(len(data)-kernel.shape[2]+1):
            for j in range(kernel.shape[2]):
                for c in range(kernel.shape[1]):
                    res[i,n] += data[i+j, c] * kernel[n,c,j]
            res[i,n] += bias[n]
    return res

def apply_convolution(data, kernel, bias):
    """
    data - InLen x InChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutLen x OutChannels
    """
    InLen = data.shape[0]                       
    KernelSize = kernel.shape[2]        
    Windows = InLen - KernelSize + 1

    sliding_indexer = np.expand_dims(np.arange(KernelSize), 0) + \
                      np.expand_dims(np.arange(Windows), 0).T   # KernelSize x Windows
    sliding_windows = data[sliding_indexer]                     # Windows x (KernelSize x InChannels)
    # для тензора транспонирование, это обратный порядок измерений:         (KernelSize x InChannels) x OutChannels
    return bias + np.tensordot(sliding_windows, kernel.T)       # -> Windows x OutChannels


data = read_array()
kernel = read_array()
bias = read_array()

result = apply_convolution(data, kernel, bias)

write_array(result)

[[4.536374007375313, 3.4055978271722807, 3.6420328514923983], [3.5379175544464094, 3.3156379278173618, 3.1978554753916804], [3.1155343252729804, 2.6461513962620473, 2.8292344624234116], [3.955826212094638, 2.6848769190617205, 2.355474927521796], [3.315456250217841, 2.5537928749371677, 2.979329283784809], [3.2335971417998324, 1.8896105910662777, 2.297264030641381], [2.5503637613247276, 2.441085171765234, 2.0663539814304355], [3.8006614791608166, 2.763727753920144, 3.030330546451793], [2.8770535772488457, 2.377839924681041, 2.6997168451820808], [3.4854598947993667, 1.9636904181843315, 1.9609925225671008], [3.3707917076650484, 2.6679105220798633, 2.2723789801177325], [4.179540206172763, 2.6157866984046363, 3.36235456621979]]


Сравним с форлупом:
- на 3 порядка (!) быстрее форлупа
- и даже немного (в 2 раза) быстрее матричного умножения
- без ложной скромности, это лучшее решение на курсе

In [10]:
scale = 10
inputs = (data, kernel, bias)

# avoid caching
rand_inputs = lambda : [np.random.random_sample([scale*s for s in arr.shape]) for arr in inputs]

In [11]:
%%timeit
apply_convolution(*rand_inputs())

2.18 ms ± 740 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%%timeit
apply_convolution_trace(*rand_inputs())

2.71 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
apply_convolution_looper(*rand_inputs())

1.86 s ± 17.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
arr = rand_inputs()
np.allclose(apply_convolution(*arr), apply_convolution_trace(*arr), apply_convolution_looper(*arr))

True

Вы применяете свёрточный модуль к данным: 
$$y = convolve(x, kernel, bias)$$ 

где 
 
$x \in \mathbb{R}^{InLen \times InChannels}$ - входная последовательность,  
$kernel \in \mathbb{R}^{OutChannels \times InChannels \times KernelSize}$ - ядро свёртки,  
$bias \in \mathbb{R}^{OutChannels}$ - параметры сдвига для каждого выходного канала.

Напишите функцию, которая находит значение производной результата по ядру свёртки: $\frac{\partial y} {\partial kernel}$

Интуитивно как то так:
- градиент это сумма тех входных данных, которые аккумулируются на соответствующих компонентах ядра
- это происходит в виде скользящего окна, поэтому вот оно
- по измерению OutLen (количество окон, т.е. первое измерение) окна схлопываются
- информация о выходных каналах исключительно в ядре, поэтому производные по разным каналам будут равны

In [15]:
SAMPLES = """[[0.1559846921787793, 0.31890243279158936, 0.4294841981370352], [0.6287087193276831, 0.041166388481120975, 0.0670771539905104], [0.15852860049868056, 0.5535509628878403, 0.7578893631796608], [0.505354018460584, 0.39104507392557886, 0.267830936523598], [0.35352084058390487, 0.09557605719492113, 0.17762289879326898], [0.17895124947325458, 0.2038143268404633, 0.45431038892117714], [0.44910520386366004, 0.28874952266426823, 0.48880576852948343], [0.978917156152191, 0.11927306276379035, 0.6831784491543058], [0.7665547409895508, 0.02661305420346527, 0.662020788170643]]\n[[0.9423069699375323, 2.235723993887441], [1.051430354504024, 2.5441990558270864], [1.3060781439427196, 2.77617352972661], [1.1097986223660692, 2.019161891632857], [0.8312656308401454, 2.1968468090151005], [1.0883546513743934, 3.182136137325698], [1.4545203460970286, 3.3294233853738975]]\n[[[0.7285150274194675, 0.13990616439149894, 0.08385531710791316], [0.8119118106104425, 0.19272155988045991, 0.010762309371285528], [0.5242309485683324, 0.33106798748722055, 0.2219888201243384]], [[0.31261137319823096, 0.3951341806652614, 0.954412244770657], [0.5475239344122861, 0.6407966544293683, 0.2840031545245296], [0.9267337670407934, 0.626334029479077, 0.4315268897320006]]]\n[0.03900532471824536, 0.6619593919342232]""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать


def calculate_kernel_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutChannels x InChannels x KernelSize
    """
    InLen = x.shape[0]               
    KernelSize = kernel.shape[2]        
    OutLen = InLen - KernelSize + 1

    sliding_indexer = np.expand_dims(np.arange(KernelSize), 0) + \
                      np.expand_dims(np.arange(OutLen), 0).T   # KernelSize x OutLen
    x_sliding = x[sliding_indexer]                             # OutLen x KernelSize x InChannels

    return np.sum(x_sliding, axis=0).T + np.zeros_like(kernel) # неявный бродкаст к нужной размерности


x = read_array()
y = read_array()
kernel = read_array()
bias = read_array()

result = calculate_kernel_grad(x, y, kernel, bias)

write_array(result)

[[[2.4301533243865463, 3.253085788359958, 3.3909318100218258], [1.8928047647857822, 1.6931753947579833, 1.6786220604803275], [2.643020708074734, 2.8967149590920043, 3.491658593272137]], [[2.4301533243865463, 3.253085788359958, 3.3909318100218258], [1.8928047647857822, 1.6931753947579833, 1.6786220604803275], [2.643020708074734, 2.8967149590920043, 3.491658593272137]]]


Вы применяете свёрточный модуль к данным: 

$$y = convolve(x, kernel, bias)$$ 


где 

$x \in \mathbb{R}^{InLen \times InChannels}$ - входная последовательность,   
$kernel \in \mathbb{R}^{OutChannels \times InChannels \times KernelSize}$ - ядро свёртки,  
$bias \in \mathbb{R}^{OutChannels}$ - параметры сдвига для каждого выходного канала.

Напишите функцию, которая находит значение производной результата по входу: $\frac{\partial y} {\partial x}$

- задача, собрать компоненты ядра, с которыми перемножается каждый вход
- на чистом numpy не получилось (ПОЛУЧИЛОСЬ)

In [16]:
SAMPLES = """[[0.5031766517322117, 0.30744410216949514], [0.04690208449415345, 0.322727131626243], [0.1388690574185909, 0.48576543724022325], [0.5260018011862109, 0.5859221562109312], [0.9194272143904142, 0.3887293155713266], [0.26873714217871125, 0.9546207791313607], [0.8974007607375208, 0.5713329992292489], [0.378989716528242, 0.49787928388753266]]\n[[1.5157583762374225, 0.9460413662192456, 0.9802340338281511], [1.5728362445918327, 0.996409724139607, 1.2530013664472253], [1.9068174476481374, 1.430592927945995, 1.6704630594015581], [2.189768979209843, 2.3149543871163503, 2.1601629609824995], [2.8353457102707083, 1.7422359297539565, 1.816707087141475], [2.0532913525958474, 1.9924093441385802, 2.3069493556139014]]\n[[[0.8077620147648772, 0.006392942850116379, 0.6080212915877307], [0.6288229869798402, 0.6410664904844843, 0.75419330562945]], [[0.5355186530459589, 0.9211024178840701, 0.27725553497982014], [0.4507098181629161, 0.081570594016668, 0.8234980185346139]], [[0.0325944131753374, 0.7744753133142763, 0.05946983249285043], [0.7059580971549311, 0.7969953841197822, 0.5257810951530107]]]\n[0.2579976950685653, 0.029957050945287222, 0.18958928880952108]""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать

def calculate_conv_x_grad_looper(x, y, kernel, bias):
    in_len = len(x)
    in_channels = len(x[0])
    out_ch = len(bias)
    kernel_size = len(kernel[0][0])
    output_len = in_len - kernel_size + 1

    result=np.zeros(x.shape)

    for pos in range(output_len):
        for oc in range(out_ch):
            for k in range(kernel_size):
                for ic in range(in_channels):
                    result[pos + k][ic] += kernel[oc][ic][k]

    return result

def all_traces_2d(arr, flip=False):
    """ на основе стаковерфлоу:
    - лучшее решение для вычисления всех следов матрицы
    """
    assert arr.ndim == 2
    if flip: arr = np.fliplr(arr)
    indexes = np.indices(arr.shape).sum(axis=0).flat
    return np.bincount(indexes, weights=arr.flat)

def calculate_conv_x_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns InLen x InChannels
    """
    InLen = x.shape[0]               
    KernelSize = kernel.shape[2]        
    OutLen = InLen - KernelSize + 1
    sliding_indexer = np.expand_dims(np.arange(KernelSize), 0) + \
                      np.expand_dims(np.arange(OutLen), 0).T   # KernelSize x OutLen
    kernel_expander = np.zeros_like(x[sliding_indexer])
    kernel_expand = np.rollaxis(kernel, -1) + np.expand_dims(kernel_expander, 1) 
    kernel_sums = kernel_expand.sum(axis=2)
    
    return np.vectorize(all_traces_2d, signature="(i,j)->(k)")(np.rollaxis(kernel_sums, -1)).T

x = read_array()
y = read_array()
kernel = read_array()
bias = read_array()

result = calculate_conv_x_grad(x, y, kernel, bias)

write_array(result)

[[1.3758750809861735, 1.7854909022976875], [3.0778457550346365, 3.305123370918622], [4.022592414095037, 5.4085957902356965], [4.022592414095037, 5.4085957902356965], [4.022592414095037, 5.4085957902356965], [4.022592414095037, 5.4085957902356965], [2.646717333108864, 3.623104887938009], [0.9447466590604012, 2.1034724193170744]]


Сравним форлуп и нампай:
- на чистейшем numpy получилось на 2 порядка быстрее (в 100 раз)
- Боже, как я хорош, срочно в продакшен

In [17]:
scale = 10
inputs = (x, y, kernel, bias)

# avoid caching
rand_inputs = lambda : [np.random.random_sample([scale*s for s in arr.shape]) for arr in inputs]

In [18]:
%%timeit
calculate_conv_x_grad(*rand_inputs())

6.01 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
calculate_conv_x_grad_looper(*rand_inputs())

630 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
arr = rand_inputs()
np.allclose(calculate_conv_x_grad(*arr), calculate_conv_x_grad_looper(*arr))

True

Вы разрабатываете нейросеть, состоящую из последовательности свёрточных слоёв с разными параметрами:

$kernel >= 1, kernel \% 2 == 1$ - размер ядра свёртки (обязательно нечётный)  
$dilation >= 1$ - коэффициент прореживания или расстояние между позициями элементов, учитываемых одной свёрткой (если dilation = 1, то это "плотная" свёртка, если dilation = 2, пропускается каждый второй элемент)  

Напишите функцию для расчёта ширины рецептивного поля - наибольшего расстояния между элементами входной последовательности (+1), которые влияют на один и тот же элемент выходной последовательности.

Получается типа:
$$r_{out} = r_{in} + (k−1) \cdot d$$

где

$r_{in}$ - ширина рецептивного поля текущего слоя  
$r_{out}$ - ширина рецептивного поля последующего слоя  
$d$ - коэффициент прореживания  
$k$ - размер ядра свёртки

In [21]:
SAMPLES = """[9, 9, 3]\n[2, 3, 4]""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать

import collections

LayerInfo = collections.namedtuple('LayerInfo', ('kernel_size', 'dilation'))

def calculate_receptive_field(layers):
    """
    layers - list of LayerInfo

    returns int - receptive field size
    """
    receptive_field = 1
    for l in layers:
        receptive_field += (l.kernel_size - 1) * l.dilation

    return receptive_field

    
kernels = read_array()
dilations = read_array()

layers = [LayerInfo(k, d) for k, d in zip(kernels, dilations)]

result = calculate_receptive_field(layers)
print(result)

49
